In [3]:
!unzip /content/drive/MyDrive/dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Y/eventroll_5773.npy    
  inflating: Y/eventroll_14.npy      
  inflating: Y/eventroll_5689.npy    
  inflating: Y/eventroll_5737.npy    
  inflating: Y/eventroll_3683.npy    
  inflating: Y/eventroll_919.npy     
  inflating: Y/eventroll_3414.npy    
  inflating: Y/eventroll_6416.npy    
  inflating: Y/eventroll_9674.npy    
  inflating: Y/eventroll_3622.npy    
  inflating: Y/eventroll_9129.npy    
  inflating: Y/eventroll_1093.npy    
  inflating: Y/eventroll_5515.npy    
  inflating: Y/eventroll_4966.npy    
  inflating: Y/eventroll_3671.npy    
  inflating: Y/eventroll_2724.npy    
  inflating: Y/eventroll_1141.npy    
  inflating: Y/eventroll_3878.npy    
  inflating: Y/eventroll_7745.npy    
  inflating: Y/eventroll_6310.npy    
  inflating: Y/eventroll_6082.npy    
  inflating: Y/eventroll_8848.npy    
  inflating: Y/eventroll_4002.npy    
  inflating: Y/eventroll_9476.npy    
  inflating: Y/eventroll_3768.npy    

In [1]:
import numpy as np
import pandas as pd
import librosa
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import IPython.display as ipd
from tensorflow.keras.optimizers import SGD
from keras import regularizers
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def eventroll_to_multihot_vector(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(11, 1000).
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return np.array(active_events)

In [5]:
l = os.listdir('/content/X')
l.sort()
filename_data = pd.DataFrame(l , columns = ['filename'])

l = os.listdir('/content/Y')
l.sort()
event_data= pd.DataFrame(l , columns = ['eventname'])

In [6]:
def r_shape(data):
  data = np.reshape(data,(64,1000))
  data =np.mean(data,axis=0)
  return data

In [7]:
data=np.zeros((10000,1000))
labels=np.zeros((10000,11))
for i,row in filename_data.iterrows():
  file_name = os.path.join(os.path.abspath('/content/X'),str(row["filename"]))
  d=np.load(file_name)
  data[i]=r_shape(d)

In [8]:
for i,row in event_data.iterrows():
  file_name = os.path.join(os.path.abspath('/content/Y'),str(row["eventname"]))
  e = np.load(file_name)
  e=eventroll_to_multihot_vector(e)
  labels[i]=e

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data,labels, test_size=0.20,random_state=42)

In [10]:
del(data)
del(labels)

In [11]:
clf = OneVsRestClassifier(SVC(max_iter=500)).fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).

In [12]:
y_pred=clf.predict(x_test)
k=f1_score(y_test, y_pred, average=None)
print(k)
k=np.mean(k,axis=0)
print(k)

[0.17905103 0.13717189 0.20602219 0.27974277 0.23959001 0.25888788
 0.24585219 0.25989673 0.8056872  0.9342246  0.23678647]
0.34390117648681257
